In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import matplotlib.patches as patches
from matplotlib.patches import Polygon 
from scipy.io import loadmat
from matplotlib.colors import ListedColormap
from matplotlib.lines import Line2D
from matplotlib.text import Text
from natsort import natsorted

## For High Power

In [2]:
power_level = 'high' #'low'
numpy_path = os.path.abspath(f'../../1_Simulation_Results/numpy_files/{power_level}_power/')

colormap_data = loadmat('../cold2hot.mat')
cold2hot = ListedColormap(colormap_data['colormap'])
hot2cold = ListedColormap(colormap_data['colormap'][::-1])

In [3]:
LIQ1 = np.load(numpy_path+'/LIQ1.npy')
FCC2 = np.load(numpy_path+'/FCC2.npy')
BCC3 = np.load(numpy_path+'/BCC3.npy')
FCC4 = np.load(numpy_path+'/FCC4.npy')
BCC5 = np.load(numpy_path+'/BCC5.npy')
temp = np.load(numpy_path+'/temp.npy')
time = np.load(numpy_path+'/time.npy')

In [4]:
time = time*1e-6 # mu_sec
time.shape

(1911,)

In [21]:
scan_speed = r'300 $\mathbf{mm/s}$'
power = r"180 $\mathbf{W}$"

def update_plot(t_step):
    
    laser_speed = 300e-3 # m/s
    scaling_factor = 1
    if time[t_step] <= 15e-6:
        laser_act_pos = 80e-6 * 1e6 # convering to meters
    else:
        laser_act_pos = (80e-6 + (time[t_step]-15e-6)*laser_speed) * 1e6  ## converting to visualization unit 
    laser_pos = laser_act_pos*scaling_factor # Laser position in accordance with figure
    
    fig, ax1 = plt.subplots(1,1, figsize=(16,8), frameon=False)
    ax1.set_facecolor((0, 0, 0, 0))
    
    # gist_rainbow
    cmap =  plt.get_cmap('Set1_r')
    cmap.set_under('white', alpha=0)
    hmap1 = ax1.imshow(LIQ1[:,:,t_step], extent=[0, 500, 0, 90], cmap=cmap, vmin=0.001, vmax=0.01, aspect=1, origin="upper", interpolation='bilinear')
    
    cmap = plt.get_cmap('rainbow_r') # ocean_r
    cmap.set_under('white', alpha=0)
    hmap2a = ax1.imshow(FCC2[:,:,t_step], extent=[0, 500, 0, 90], cmap=cmap, vmin=0.4, vmax=0.6, aspect=1, origin="upper", interpolation='bilinear')
    cmap = plt.get_cmap('rainbow_r') # RdYlGn
    cmap.set_under('white', alpha=0)
    hmap2b = ax1.imshow(FCC4[:,:,t_step], extent=[0, 500, 0, 90], cmap=cmap, vmin=0.01, vmax=0.05, aspect=1, origin="upper", interpolation='bilinear')
    
    cmap = plt.get_cmap('Pastel1_r') # Pastel1_r
    cmap.set_under('white', alpha=0) 
    hmap3a = ax1.imshow(BCC3[:,:,t_step], extent=[0, 500, 0, 90], cmap=cmap, vmin=0.4, vmax=0.6, aspect=1, origin="upper", interpolation='bilinear')
    
    cmap = plt.get_cmap('Pastel1_r') # winter
    cmap.set_under('white', alpha=0)
    hmap3b = ax1.imshow(BCC5[:,:,t_step], extent=[0, 500, 0, 90], cmap=cmap, vmin=0.4, vmax=0.6, aspect=1, origin="upper", interpolation='bilinear')
    
    contour_levels = [677, 1468,  1864, ]
    label_colors = ['k', 'k', 'cyan', 'white']
    contour = ax1.contour(temp[:, :, t_step], extent=[0, 500, 0, 90], levels=contour_levels, linewidths=5, origin="upper", colors='orange', linestyles='dashdot')
    [(label.set_fontweight('bold'), label.set_color(color)) for label, color in zip(plt.clabel(contour, inline=True, fontsize=16, fmt='%1.0f K'), label_colors)]
    
    # ax1.arrow(laser_pos, -51, 0, 50,  width = 4.5, color='red', length_includes_head=True, clip_on=False)
    # ax1.text(laser_pos-20,-12, power, rotation=90, color='k', fontsize=20, weight='bold')
            
    # ax1.set_title(f't = {time[t_step]*1e6:.0f} '+r'$\mathbf{\mu}$s', pad=10, loc='right', fontsize=25, weight='bold')

    if laser_pos < 250:
        ax1.set_title(f't = {time[t_step]*1e6:.0f} '+r'$\mathbf{\mu}$s', pad=10, loc='right', fontsize=25, weight='bold')
    else:
        ax1.set_title(f't = {time[t_step]*1e6:.0f} '+r'$\mathbf{\mu}$s', pad=10, loc='left', fontsize=25, weight='bold')        
    
    # ax1.text(laser_pos+10, -40, s=f'{scan_speed}', fontsize=20, weight = 'bold')
    # ax1.arrow(laser_pos+10, -28, 30, 0,  width = 3, color='k', fill=False, clip_on=False)

    if laser_pos <= 415.5:
        ax1.text(laser_pos+10, -40, s=f'{scan_speed}', fontsize=20, weight = 'bold')
        ax1.arrow(laser_pos+10, -28, 30, 0,  width = 3, color='k', fill=False, clip_on=False)
        ax1.arrow(laser_pos, -51, 0, 50,  width = 4.5, color='red', length_includes_head=True, clip_on=False)
        ax1.text(laser_pos-20,-12, power, rotation=90, color='k', fontsize=20, weight='bold')
    else:
        ax1.text(400, -20, s=f'Laser Off', fontsize=20, weight = 'bold')
        ax1.arrow(laser_pos, -51, 0, 50,  width = 4.5, color='none', length_includes_head=True, clip_on=False)
        
    
    ax1.set_xlim(-1, 500)
    ax1.set_ylim(91, -1)
    
    ax1.set_xlabel('$\mathbf{\longleftarrow}$                 500 $ \mathbf{\mu m}$                $\mathbf{\longrightarrow}$', fontsize = 27, weight ='bold',)
    ax1.annotate(r'$\mathbf{\leftarrow}$90 $\mathbf{\mu m}$ $\mathbf{\rightarrow}$', xy=(0.5, 0.5), xytext=(-0.025, 0.5), rotation=90,
                 xycoords='axes fraction', textcoords='axes fraction', fontsize = 27, weight = 'bold', color='k', ha='center', va='center')
    
    ax1.tick_params(axis='both', labelcolor='black', labelsize=65, bottom=False, top=False, left=False, right=False, labelbottom=False, labelleft=False)
    ax1.spines[:].set_linewidth(2)
    
    FCC2_txt = ax1.text(450, 20, r'$\mathbf{\gamma_{G_1}}$', fontsize = 40, weight='bold', color='white')
    FCC4_txt = ax1.text(250, 65, r'$\mathbf{\gamma_{G_2}}$', fontsize = 40, weight='bold', color='white')
    BCC3_txt = ax1.text(15, 20, r'$\mathbf{\alpha_{G_1}}$', fontsize = 40, weight='bold', color='k')
    BCC5_txt = ax1.text(125, 87, r'$\mathbf{\alpha_{G_2}}$', fontsize = 30, weight='bold', color='k')

# t_steps = 100 # 1350
# ############ To save the image as well #############
for t_step in range(1100, 1911, 1):
# for t_step in range(t_steps):
    if f'{time[t_step]*1e6:.0f}' != f'{time[t_step-1]*1e6:.0f}':
        update_plot(t_step)
        plt.savefig(f'../temperoray_figures/{power_level}_power/phase_evolve/f_{t_step}.png',bbox_inches='tight', dpi=100)
    plt.close()
print("FINISHED")

FINISHED


In [22]:
## Remaing the image names for sequential

input_dir = f'../temperoray_figures/{power_level}_power/phase_evolve/'
files = natsorted([f for f in os.listdir(input_dir) if f.startswith("f_") and f.endswith(".png")])

for i, fname in enumerate(files, start=0):
    old_path = os.path.join(input_dir, fname)
    new_path = os.path.join(input_dir, f"f_{i}.png")
    os.rename(old_path, new_path)
print("Finished")

Finished


## For Lower Power

In [23]:
power_level = 'low'
numpy_path = os.path.abspath(f'../../1_Simulation_Results/numpy_files/{power_level}_power/')

colormap_data = loadmat('../cold2hot.mat')
cold2hot = ListedColormap(colormap_data['colormap'])
hot2cold = ListedColormap(colormap_data['colormap'][::-1])

In [24]:
LIQ1 = np.load(numpy_path+'/LIQ1.npy')
FCC2 = np.load(numpy_path+'/FCC2.npy')
BCC3 = np.load(numpy_path+'/BCC3.npy')
FCC4 = np.load(numpy_path+'/FCC4.npy')
BCC5 = np.load(numpy_path+'/BCC5.npy')
temp = np.load(numpy_path+'/temp.npy')
time = np.load(numpy_path+'/time.npy')

In [25]:
time = time*1e-6 # mu_sec
time.shape

(633,)

In [26]:
scan_speed = r'300 $\mathbf{mm/s}$'
power = r"45 $\mathbf{W}$"

def update_plot(t_step):
    
    laser_speed = 300e-3 # m/s
    scaling_factor = 1
    if time[t_step] <= 15e-6:
        laser_act_pos = 80e-6 * 1e6 # convering to meters
    else:
        laser_act_pos = (80e-6 + (time[t_step]-15e-6)*laser_speed) * 1e6  ## converting to visualization unit 
    laser_pos = laser_act_pos*scaling_factor # Laser position in accordance with figure
    
    fig, ax1 = plt.subplots(1,1, figsize=(16,8), frameon=False)
    ax1.set_facecolor((0, 0, 0, 0))
    
    # gist_rainbow
    cmap =  plt.get_cmap('Set1_r')
    cmap.set_under('white', alpha=0)
    hmap1 = ax1.imshow(LIQ1[:,:,t_step], extent=[0, 500, 0, 90], cmap=cmap, vmin=0.001, vmax=0.01, aspect=1, origin="upper", interpolation='bilinear')
    
    cmap = plt.get_cmap('rainbow_r') # ocean_r
    cmap.set_under('white', alpha=0)
    hmap2a = ax1.imshow(FCC2[:,:,t_step], extent=[0, 500, 0, 90], cmap=cmap, vmin=0.4, vmax=0.6, aspect=1, origin="upper", interpolation='bilinear')
    cmap = plt.get_cmap('rainbow_r') # RdYlGn
    cmap.set_under('white', alpha=0)
    hmap2b = ax1.imshow(FCC4[:,:,t_step], extent=[0, 500, 0, 90], cmap=cmap, vmin=0.01, vmax=0.05, aspect=1, origin="upper", interpolation='bilinear')
    
    cmap = plt.get_cmap('Pastel1_r') # Pastel1_r
    cmap.set_under('white', alpha=0) 
    hmap3a = ax1.imshow(BCC3[:,:,t_step], extent=[0, 500, 0, 90], cmap=cmap, vmin=0.4, vmax=0.6, aspect=1, origin="upper", interpolation='bilinear')
    
    cmap = plt.get_cmap('Pastel1_r') # winter
    cmap.set_under('white', alpha=0)
    hmap3b = ax1.imshow(BCC5[:,:,t_step], extent=[0, 500, 0, 90], cmap=cmap, vmin=0.4, vmax=0.6, aspect=1, origin="upper", interpolation='bilinear')
    
    contour_levels = [677, 1468,  1864, ]
    label_colors = ['k', 'k', 'cyan', 'white']
    contour = ax1.contour(temp[:, :, t_step], extent=[0, 500, 0, 90], levels=contour_levels, linewidths=5, origin="upper", colors='orange', linestyles='dashdot')
    [(label.set_fontweight('bold'), label.set_color(color)) for label, color in zip(plt.clabel(contour, inline=True, fontsize=16, fmt='%1.0f K'), label_colors)]
               
    if laser_pos < 250:
        ax1.set_title(f't = {time[t_step]*1e6:.0f} '+r'$\mathbf{\mu}$s', pad=10, loc='right', fontsize=25, weight='bold')
    else:
        ax1.set_title(f't = {time[t_step]*1e6:.0f} '+r'$\mathbf{\mu}$s', pad=10, loc='left', fontsize=25, weight='bold')            

    if laser_pos <= 415.5:
        ax1.text(laser_pos+10, -40, s=f'{scan_speed}', fontsize=20, weight = 'bold')
        ax1.arrow(laser_pos+10, -28, 30, 0,  width = 3, color='k', fill=False, clip_on=False)
        ax1.arrow(laser_pos, -51, 0, 50,  width = 4.5, color='red', length_includes_head=True, clip_on=False)
        ax1.text(laser_pos-20,-12, power, rotation=90, color='k', fontsize=20, weight='bold')
    else:
        ax1.text(400, -20, s=f'Laser Off', fontsize=20, weight = 'bold')
        ax1.arrow(laser_pos, -51, 0, 50,  width = 4.5, color='none', length_includes_head=True, clip_on=False)
        
    
    ax1.set_xlim(-1, 500)
    ax1.set_ylim(91, -1)
    
    ax1.set_xlabel('$\mathbf{\longleftarrow}$                 500 $ \mathbf{\mu m}$                $\mathbf{\longrightarrow}$', fontsize = 27, weight ='bold',)
    ax1.annotate(r'$\mathbf{\leftarrow}$90 $\mathbf{\mu m}$ $\mathbf{\rightarrow}$', xy=(0.5, 0.5), xytext=(-0.025, 0.5), rotation=90,
                 xycoords='axes fraction', textcoords='axes fraction', fontsize = 27, weight = 'bold', color='k', ha='center', va='center')
    
    ax1.tick_params(axis='both', labelcolor='black', labelsize=65, bottom=False, top=False, left=False, right=False, labelbottom=False, labelleft=False)
    ax1.spines[:].set_linewidth(2)
    
    FCC2_txt = ax1.text(450, 20, r'$\mathbf{\gamma_{G_1}}$', fontsize = 40, weight='bold', color='white')
    FCC4_txt = ax1.text(250, 65, r'$\mathbf{\gamma_{G_2}}$', fontsize = 40, weight='bold', color='white')
    BCC3_txt = ax1.text(15, 20, r'$\mathbf{\alpha_{G_1}}$', fontsize = 40, weight='bold', color='k')
    BCC5_txt = ax1.text(125, 87, r'$\mathbf{\alpha_{G_2}}$', fontsize = 30, weight='bold', color='k')

#569 Laser is off here
# ############ To save the image as well #############
for t_step in range(550, 633, 1):
# for t_step in range(t_steps):
    if f'{time[t_step]*1e6:.0f}' != f'{time[t_step-1]*1e6:.0f}':
        update_plot(t_step)
        plt.savefig(f'../temperoray_figures/{power_level}_power/phase_evolve/f_{t_step}.png',bbox_inches='tight', dpi=100)
    plt.close()
print("FINISHED")

FINISHED


In [27]:
## Remaing the image names for sequential

input_dir = f'../temperoray_figures/{power_level}_power/phase_evolve/'
files = natsorted([f for f in os.listdir(input_dir) if f.startswith("f_") and f.endswith(".png")])

for i, fname in enumerate(files, start=0):
    old_path = os.path.join(input_dir, fname)
    new_path = os.path.join(input_dir, f"f_{i}.png")
    os.rename(old_path, new_path)
print("Finished")

Finished
